### Importing libraries

In [31]:
from models import Yolov4
model = Yolov4(weight_path='yolov4.weights',
               class_name_path='class_names/coco_classes.txt')

import pandas as pd
import cv2
import numpy as np
import easyocr
from PIL import Image, ImageDraw
import docker
%matplotlib inline
import matplotlib.pyplot as plt
import math
import os

nms iou: 0.413 score: 0.3
all weights read
load from yolov4.weights


### Loading dataframe from CSV

In [32]:
features = ['time','frame_name','frame_name_2','lat','lon','hmsl','ggs','gc','hacc','satellite','ping_edge','ping_six','iperf_down','iperf_up','netmode']


df = pd.read_csv('data/2023-01-07_16_10_24.686639.csv', skiprows=2652, nrows=20, header=None, names=features)

factor = 5

In [90]:
df.head()

,time,frame_name,frame_name_2,lat,lon,hmsl,ggs,gc,hacc,satellite,ping_edge,ping_six,iperf_down,iperf_up,netmode
0,2023-01-07 16:13:59.039037,frame_1_6566.jpg,NaN,48.141312,17.075224,161384.4,2462,7.07284,14.1,51,22.998,30.162,7649,44361,5G-NR
1,2023-01-07 16:13:59.067829,frame_1_6567.jpg,NaN,48.141312,17.075224,161384.4,2462,7.07284,14.1,51,22.998,30.162,7649,44361,5G-NR
2,2023-01-07 16:13:59.100841,frame_1_6568.jpg,NaN,48.141312,17.075224,161384.4,2462,7.07284,14.1,51,22.998,30.162,7649,44361,5G-NR
3,2023-01-07 16:13:59.142974,frame_1_6569.jpg,NaN,48.141312,17.075224,161384.4,2462,7.07284,14.1,51,22.998,30.162,7649,44361,5G-NR
4,2023-01-07 16:13:59.168048,frame_1_6570.jpg,NaN,48.141312,17.075224,161384.4,2462,7.07284,14.1,51,22.998,30.162,7649,44361,5G-NR


### Image preprocessing functions

In [35]:
def increase_contrast(image, alpha=1.5, beta=10):
    result = np.zeros(image.shape, dtype=np.uint8)
    
    cv2.convertScaleAbs(image, result, alpha=alpha, beta=beta)
    
    return result

def preprocess_image(image):
    
    height, width = image.shape[:2]
        
    image = cv2.resize(image, (int(width * factor), int(height * factor)), interpolation=cv2.INTER_LANCZOS4)
    
    image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15)
    
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    norm_img = np.zeros((image.shape[0], image.shape[1]))
    # Image normalization
    image = cv2.normalize(gray_image, norm_img, 0, 255, cv2.NORM_MINMAX)
    # Increasing contrast for better readability
    image = increase_contrast(image)
    # Removing noise from color image
    #image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15)
    
    kernel = np.ones((5, 5), np.uint8)
    
    image = cv2.erode(image, kernel, iterations=1)
    
    kernel = np.ones((5, 5), np.uint8)
    # Aplikovanie dilatácie na obrázok
    image = cv2.dilate(image, kernel, iterations=1)
    
    #gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Image segmentation
    #_, result = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    #image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 21, 2)
    
    return image

def is_negative(num):
    return num if num >= 0 else 0

In [179]:
def enhance_contrast(image):
    ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    channels = cv2.split(ycrcb_image)
    cv2.equalizeHist(channels[0], channels[0])
    enhanced_image = cv2.merge(channels)
    return cv2.cvtColor(enhanced_image, cv2.COLOR_YCrCb2BGR)

### Blurring license plates
- Detection of cars in the image
- The use of the model to recognize license plates 
- The use of easyocr for text detection in the image

'D:\\skola\\AIL\\yolo'

In [19]:
def split_func(x):
    return int(x.split('_1_')[1])

In [21]:
file_names = [os.path.splitext(file)[0] for file in os.listdir()]
fn = sorted(file_names, key=split_func)
fn[:5]

['frame_1_13', 'frame_1_14', 'frame_1_15', 'frame_1_16', 'frame_1_17']

In [42]:
reader = easyocr.Reader(['en'])
model1 = cv2.CascadeClassifier('haarcascade_russian_plate_number.xml')

# Iteration through the dataframe
#for index, row in df.iterrows():
for f in fn[43:]:
        
    #file_name = 'img/' + row['frame_name']
    file_name = 'img/' + f + '.jpg'
    print(file_name)
    
    # Object prediction with YoLo 
    predictions = model.predict(file_name)

    # Filtering only vehicles
    cars = predictions.loc[(predictions['class_name'].isin(['car', 'bus', 'truck'])) & (predictions['w'] > 50) & (predictions['h'] > 50)]
    people = predictions.loc[(predictions['class_name'].isin(['person']))]
    image = cv2.imread(file_name)
    
    # Iterate over individual vehicles
    for idx, car in cars.iterrows():
        # Cutting out the vehicle from the image
        c_x, c_y, c_h, c_w = car['x1'], car['y1'], car['h'], car['w']
        enlarged_car = cv2.resize(image[c_y:c_y + c_h, c_x:c_x + c_w], (factor*c_w, factor*c_h), interpolation=cv2.INTER_LANCZOS4)
    
        plates = model1.detectMultiScale(enlarged_car, scaleFactor=1.01, minNeighbors=6, flags=cv2.CASCADE_DO_CANNY_PRUNING)
        
        for (x, y, w, h) in plates:
            
            if w > enlarged_car.shape[1]//2:
                continue
            car_roi = enlarged_car[y:y+h, x:x+w]
            
            blur = cv2.GaussianBlur(car_roi, (99, 99), 100)
            enlarged_car[y:y+h, x:x+w] = blur
            
            #cv2.rectangle(enlarged_car, (x, y), (x + w, y + h), (0, 0, 0), -1)
            #image[c_y:c_y + c_h, c_x:c_x + c_w] = cropped_image
        
        cropped_image = cv2.resize(enlarged_car, (c_w, c_h))
    
        # Vloženie upraveného obrázka späť do pôvodného obrázka
        image[c_y:c_y + c_h, c_x:c_x + c_w] = cropped_image

        # Car image preprocessing
        preprocessed_image = preprocess_image(cropped_image)
    
        #plt.imshow(cv2.cvtColor(preprocessed_image, cv2.COLOR_BGR2RGB))
        #plt.show()
        #cv2.imwrite('new_img/ahoj' + str(idx) + '.jpg', preprocessed_image)
        # Using easyocr to search for text on car image
        result = reader.readtext(preprocessed_image, detail=1, paragraph=False)
        
        # Iteration if there is more than one detected text in the car
        for text in result:
            crd = text[0]
            t_x1, t_y1 = int(is_negative(crd[0][0])) / factor, int(is_negative(crd[0][1])) / factor
            t_x2, t_y2 = int(is_negative(crd[2][0])) / factor, int(is_negative(crd[2][1])) / factor

            t_w = abs(math.ceil(t_x2 - t_x1))
            t_h = abs(math.ceil(t_y2 - t_y1))
            
            if t_w * factor > preprocessed_image.shape[1] // 2 or t_w == 0 or t_h == 0:
                continue
                
            main_text_x = math.ceil(c_x + t_x1)
            main_text_y = math.ceil(c_y + t_y1)
            
            
            blur = cv2.GaussianBlur(image[main_text_y:main_text_y + t_h, main_text_x:main_text_x+t_w], (99, 99), 100)
            image[main_text_y:main_text_y + t_h, main_text_x:main_text_x + t_w] = blur 
            
        # If there was no found and car width and height is bigger than 100, than log filename.
        if(len(result) == 0 and len(plates) == 0 and c_w >= 100 and c_h >= 100):
            print(file_name)
            print('Na aute nebola detegovana EČV')
    
    # Iteration over every person found in the image
    for idx, person in people.iterrows():
        
        p_x, p_y, p_h, p_w = person['x1'], person['y1'], person['h'], person['w']
        prsn_img = image[p_y:p_y + p_h, p_x:p_x + p_w] 
        blur = cv2.GaussianBlur(prsn_img, (25, 25), 30)
        
        image[p_y:p_y+p_h, p_x:p_x+p_w] = blur
    
    cv2.imwrite('new_img/' + f + '.jpg', image)


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


img/frame_1_56.jpg
1/1 [==============================] - 0s 486ms/step
# of bboxes: 7
img/frame_1_57.jpg
1/1 [==============================] - 1s 666ms/step
# of bboxes: 7
img/frame_1_58.jpg
1/1 [==============================] - 1s 642ms/step
# of bboxes: 8
img/frame_1_59.jpg
1/1 [==============================] - 1s 600ms/step
# of bboxes: 7
img/frame_1_60.jpg
1/1 [==============================] - 0s 472ms/step
# of bboxes: 5
img/frame_1_61.jpg
1/1 [==============================] - 1s 518ms/step
# of bboxes: 6
img/frame_1_62.jpg
1/1 [==============================] - 1s 579ms/step
# of bboxes: 8
img/frame_1_63.jpg
1/1 [==============================] - 1s 611ms/step
# of bboxes: 10
img/frame_1_64.jpg
1/1 [==============================] - 1s 518ms/step
# of bboxes: 11
img/frame_1_65.jpg
1/1 [==============================] - 1s 792ms/step
# of bboxes: 14
img/frame_1_66.jpg
1/1 [==============================] - 1s 646ms/step
# of bboxes: 12
img/frame_1_66.jpg
Na aute nebola deteg

1/1 [==============================] - 1s 602ms/step
# of bboxes: 4
img/frame_1_282.jpg
1/1 [==============================] - 1s 544ms/step
# of bboxes: 5
img/frame_1_283.jpg
1/1 [==============================] - 1s 542ms/step
# of bboxes: 5
img/frame_1_284.jpg
1/1 [==============================] - 1s 555ms/step
# of bboxes: 5
img/frame_1_285.jpg
1/1 [==============================] - 1s 544ms/step
# of bboxes: 5
img/frame_1_286.jpg
1/1 [==============================] - 1s 634ms/step
# of bboxes: 4
img/frame_1_287.jpg
1/1 [==============================] - 1s 584ms/step
# of bboxes: 4
img/frame_1_288.jpg
1/1 [==============================] - 1s 559ms/step
# of bboxes: 5
img/frame_1_289.jpg
1/1 [==============================] - 1s 550ms/step
# of bboxes: 6
img/frame_1_290.jpg
1/1 [==============================] - 1s 551ms/step
# of bboxes: 5
img/frame_1_291.jpg
1/1 [==============================] - 1s 549ms/step
# of bboxes: 6
img/frame_1_292.jpg
1/1 [===========================

1/1 [==============================] - 1s 811ms/step
# of bboxes: 7
img/frame_1_370.jpg
1/1 [==============================] - 1s 783ms/step
# of bboxes: 6
img/frame_1_371.jpg
1/1 [==============================] - 1s 552ms/step
# of bboxes: 7
img/frame_1_372.jpg
1/1 [==============================] - 1s 558ms/step
# of bboxes: 6
img/frame_1_373.jpg
1/1 [==============================] - 1s 595ms/step
# of bboxes: 7
img/frame_1_374.jpg
1/1 [==============================] - 1s 828ms/step
# of bboxes: 8
img/frame_1_375.jpg
1/1 [==============================] - 1s 859ms/step
# of bboxes: 7
img/frame_1_376.jpg
1/1 [==============================] - 1s 652ms/step
# of bboxes: 7
img/frame_1_377.jpg
1/1 [==============================] - 1s 738ms/step
# of bboxes: 8
img/frame_1_378.jpg
1/1 [==============================] - 1s 562ms/step
# of bboxes: 9
img/frame_1_379.jpg
1/1 [==============================] - 1s 631ms/step
# of bboxes: 6
img/frame_1_380.jpg
1/1 [===========================

# of bboxes: 10
img/frame_1_605.jpg
1/1 [==============================] - 1s 516ms/step
# of bboxes: 11
img/frame_1_606.jpg
1/1 [==============================] - 1s 500ms/step
# of bboxes: 7
img/frame_1_607.jpg
1/1 [==============================] - 1s 516ms/step
# of bboxes: 8
img/frame_1_608.jpg
1/1 [==============================] - 0s 453ms/step
# of bboxes: 8
img/frame_1_609.jpg
1/1 [==============================] - 1s 514ms/step
# of bboxes: 11
img/frame_1_610.jpg
1/1 [==============================] - 1s 516ms/step
# of bboxes: 9
img/frame_1_611.jpg
1/1 [==============================] - 1s 502ms/step
# of bboxes: 8
img/frame_1_612.jpg
1/1 [==============================] - 1s 516ms/step
# of bboxes: 12
img/frame_1_613.jpg
1/1 [==============================] - 1s 516ms/step
# of bboxes: 9
img/frame_1_614.jpg
1/1 [==============================] - 1s 564ms/step
# of bboxes: 10
img/frame_1_615.jpg
1/1 [==============================] - 0s 500ms/step
# of bboxes: 8
img/frame_1_

...

1/1 [==============================] - 1s 594ms/step
# of bboxes: 16
img/frame_1_45360.jpg
1/1 [==============================] - 1s 500ms/step
# of bboxes: 14
img/frame_1_46105.jpg
1/1 [==============================] - 0s 500ms/step
# of bboxes: 3
img/frame_1_46106.jpg
1/1 [==============================] - 0s 500ms/step
# of bboxes: 6
img/frame_1_46107.jpg
1/1 [==============================] - 0s 468ms/step
# of bboxes: 6
img/frame_1_46108.jpg
1/1 [==============================] - 1s 594ms/step
# of bboxes: 5
img/frame_1_46109.jpg
1/1 [==============================] - 1s 516ms/step
# of bboxes: 5
img/frame_1_46110.jpg
1/1 [==============================] - 1s 594ms/step
# of bboxes: 7
img/frame_1_47485.jpg
1/1 [==============================] - 1s 1s/step
# of bboxes: 12
img/frame_1_47486.jpg
1/1 [==============================] - 98s 98s/step
# of bboxes: 14
img/frame_1_47486.jpg
Na aute nebola detegovana EČV
img/frame_1_47487.jpg
1/1 [==============================] - 19s 19s/s

1/1 [==============================] - 1s 595ms/step
# of bboxes: 13
img/frame_1_60190.jpg
1/1 [==============================] - 1s 606ms/step
# of bboxes: 13
img/frame_1_60191.jpg
1/1 [==============================] - 1s 589ms/step
# of bboxes: 11
img/frame_1_60192.jpg
1/1 [==============================] - 1s 638ms/step
# of bboxes: 12
img/frame_1_60193.jpg
1/1 [==============================] - 1s 570ms/step
# of bboxes: 15
img/frame_1_60194.jpg
1/1 [==============================] - 1s 558ms/step
# of bboxes: 12
img/frame_1_60195.jpg
1/1 [==============================] - 1s 506ms/step
# of bboxes: 12
img/frame_1_60196.jpg
1/1 [==============================] - 0s 478ms/step
# of bboxes: 13
img/frame_1_60197.jpg
1/1 [==============================] - 1s 503ms/step
# of bboxes: 14
img/frame_1_60198.jpg
1/1 [==============================] - 1s 599ms/step
# of bboxes: 11
img/frame_1_60199.jpg
1/1 [==============================] - 1s 570ms/step
# of bboxes: 11
img/frame_1_60200.jpg

1/1 [==============================] - 1s 646ms/step
# of bboxes: 18
img/frame_1_63595.jpg
1/1 [==============================] - 1s 635ms/step
# of bboxes: 19
img/frame_1_63596.jpg
1/1 [==============================] - 1s 633ms/step
# of bboxes: 19
img/frame_1_63597.jpg
1/1 [==============================] - 1s 708ms/step
# of bboxes: 20
img/frame_1_63598.jpg
1/1 [==============================] - 1s 633ms/step
# of bboxes: 17
img/frame_1_63599.jpg
1/1 [==============================] - 1s 611ms/step
# of bboxes: 21
img/frame_1_63600.jpg
1/1 [==============================] - 1s 619ms/step
# of bboxes: 20
img/frame_1_63601.jpg
1/1 [==============================] - 1s 618ms/step
# of bboxes: 20
img/frame_1_63602.jpg
1/1 [==============================] - 1s 601ms/step
# of bboxes: 19
img/frame_1_63603.jpg
1/1 [==============================] - 1s 634ms/step
# of bboxes: 19
img/frame_1_63604.jpg
1/1 [==============================] - 1s 597ms/step
# of bboxes: 21
img/frame_1_63605.jpg

1/1 [==============================] - 1s 590ms/step
# of bboxes: 3
img/frame_1_72135.jpg
1/1 [==============================] - 1s 604ms/step
# of bboxes: 3
img/frame_1_72136.jpg
1/1 [==============================] - 0s 482ms/step
# of bboxes: 2
img/frame_1_72137.jpg
1/1 [==============================] - 1s 608ms/step
# of bboxes: 2
img/frame_1_72138.jpg
1/1 [==============================] - 0s 483ms/step
# of bboxes: 3
img/frame_1_72139.jpg
1/1 [==============================] - 0s 498ms/step
# of bboxes: 3
img/frame_1_72140.jpg
1/1 [==============================] - 1s 535ms/step
# of bboxes: 3
img/frame_1_72141.jpg
1/1 [==============================] - 1s 597ms/step
# of bboxes: 3
img/frame_1_72142.jpg
1/1 [==============================] - 1s 505ms/step
# of bboxes: 3
img/frame_1_72143.jpg
1/1 [==============================] - 1s 509ms/step
# of bboxes: 3
img/frame_1_72144.jpg
1/1 [==============================] - 1s 591ms/step
# of bboxes: 3
img/frame_1_72145.jpg
1/1 [=====

1/1 [==============================] - 1s 637ms/step
# of bboxes: 1
img/frame_1_72672.jpg
1/1 [==============================] - 1s 590ms/step
# of bboxes: 1
img/frame_1_72673.jpg
1/1 [==============================] - 1s 552ms/step
# of bboxes: 1
img/frame_1_72674.jpg
1/1 [==============================] - 1s 658ms/step
# of bboxes: 1
img/frame_1_72675.jpg
1/1 [==============================] - 0s 494ms/step
# of bboxes: 1
